In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from collections import defaultdict

plt.rcParams["figure.figsize"] = (15,8) #set size of plot

Cell used for scraping of calories:

In [ ]:
#Get the URL
URL = 'https://health.gov/dietaryguidelines/2015/guidelines/appendix-2/#males'
r = get(URL)
page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser');

Loading of USDA Food Database

In [ ]:
usda_foods = pd.read_excel("data/USDA-Food.xlsx", sheet_name=0)
usda_foods = usda_foods.drop(columns=['Database Number'])
test = usda_foods[['Food Group', 'Food Name', 'Protein (g)', 'Carbohydrates (g)', 'Fat (g)']]